# UTC is the db timezone for the connection. So I need to convert from UTC to that States Timezone.
- find_tz - function find the time zone of the State provided - Location_State -> returns Location_State_TZ
- convert_tz function - converts time zone from UTC_TZ to Location_State_TZ

This is essentially a really large case statement. Not the best performant but if you have the will to find a solution, you can using LookML. But what I am offering is much more elegant. 
The US Package even have shapefiles which I dont have a use case yet but will dive into it in a post: https://pypi.org/project/us/

There are 2 options in US, capital_tz and time_zones. Time_zones can be mulyiple per state. Can not use the this: zone=us.states.lookup('OR').time_zones
But will use capital_tz to return only one. I tested this with OR. 

Oregon and Idaho are split between the Mountain and Pacific time zones. Nebraska, Kansas, Texas, North and South Dakota are divided between Central and Mountain time zones. Florida, Michigan, Indiana, Kentucky, and Tennessee are split between Eastern and Central time zones. (https://www.usatoday.com/picture-gallery/travel/news/2020/02/05/us-time-zone-map-texas-alaska-florida-oregon-have-two/41074419/)

In [13]:
from datetime import datetime
import pytz # Wont work in snwoflake therefor eneed to use build in timezone. mentionin Medium article
import us

In [4]:
import pytz

tznames = pytz.country_timezones['us']
tznames

['America/New_York',
 'America/Detroit',
 'America/Kentucky/Louisville',
 'America/Kentucky/Monticello',
 'America/Indiana/Indianapolis',
 'America/Indiana/Vincennes',
 'America/Indiana/Winamac',
 'America/Indiana/Marengo',
 'America/Indiana/Petersburg',
 'America/Indiana/Vevay',
 'America/Chicago',
 'America/Indiana/Tell_City',
 'America/Indiana/Knox',
 'America/Menominee',
 'America/North_Dakota/Center',
 'America/North_Dakota/New_Salem',
 'America/North_Dakota/Beulah',
 'America/Denver',
 'America/Boise',
 'America/Phoenix',
 'America/Los_Angeles',
 'America/Anchorage',
 'America/Juneau',
 'America/Sitka',
 'America/Metlakatla',
 'America/Yakutat',
 'America/Nome',
 'America/Adak',
 'Pacific/Honolulu']

In [14]:
#do not use
zone=us.states.lookup('OR').time_zones
print(zone)
print(type(zone))

['America/Los_Angeles', 'America/Boise']
<class 'list'>


In [15]:
location_state = 'OR'
us.states.lookup(location_state).capital_tz


'America/Los_Angeles'

In [4]:
#find_tz - function find the time zone of the State provided - Location_State -> returns Location_State_TZ
def find_tz(Location_State):
    Location_State_TZ = us.states.lookup(Location_State).capital_tz
    return Location_State_TZ

Location_State = 'OR'
print(find_tz(Location_State))

America/Los_Angeles


In [ ]:
def convert_datetime_timezone(dt, tz1, tz2):
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)

    dt = datetime.datetime.strptime(dt,"%Y-%m-%d %H:%M:%S")
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    dt = dt.strftime("%Y-%m-%d %H:%M:%S")



In [10]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz
utc = pytz.utc
utc_dt = utc.localize(datetime.utcfromtimestamp(1143408899))

utc.zone

'UTC'

In [11]:
eastern = timezone('America/Los_Angeles')
eastern.zone

'US/Eastern'

In [13]:
amsterdam = timezone('Europe/Amsterdam')
fmt = '%Y-%m-%d %H:%M:%S %Z%z'

au_tz = timezone('Australia/Sydney')
au_dt = utc_dt.astimezone(timezone('Australia/Sydney'))
print(au_dt.strftime(fmt))

2006-03-27 08:34:59 AEDT+1100


# With pytz and us

In [1]:
from datetime import datetime
import pytz
import us

def find_tz(location_state_abbr):
    state = us.states.lookup(location_state_abbr)
    if state:
        location_state_tz = pytz.timezone(state.capital_tz)
        return location_state_tz
    else:
        return None

def time_hopper(location_state_abbr, signing_time_str):
    signing_time = datetime.strptime(signing_time_str, '%Y-%m-%d %H:%M')
    state_timezone = find_tz(location_state_abbr)
    if state_timezone:
        signing_time_utc = signing_time.replace(tzinfo=pytz.utc)
        converted_time = signing_time_utc.astimezone(state_timezone)
        return converted_time
    else:
        return None

if __name__ == '__main__':
    location_state_abbr = 'CA'
    signing_time_str = '2023-02-17 17:30'
    converted_time = time_hopper(location_state_abbr, signing_time_str)
    
    if converted_time:
        print(f'Original time (UTC): {signing_time_str}')
        print(f'Converted time ({location_state_abbr}): {converted_time}')
    else:
        print('Invalid state abbreviation.')


Original time (UTC): 2023-02-17 17:30
Converted time (CA): 2023-02-17 09:30:00-08:00


# Without PyTZ

In [5]:
from datetime import datetime
from dateutil import tz
import us

def find_tz(location_state_abbr):
    state = us.states.lookup(location_state_abbr)
    if state:
        location_state_tz = tz.gettz(state.capital_tz)
        return location_state_tz
    else:
        return None

def time_hopper(location_state_abbr, signing_time_str):
    signing_time = datetime.strptime(signing_time_str, '%Y-%m-%d %H:%M')
    state_timezone = find_tz(location_state_abbr)
    if state_timezone:
        signing_time_utc = signing_time.replace(tzinfo=tz.UTC)
        converted_time = signing_time_utc.astimezone(state_timezone)
        return converted_time
    else:
        return None

if __name__ == '__main__':
    location_state_abbr = 'CA'
    signing_time_str = '2023-02-17 17:30'
    converted_time = time_hopper(location_state_abbr, signing_time_str)
    
    if converted_time:
        print(f'Original time (UTC): {signing_time_str}')
        print(f'Converted time ({location_state_abbr}): {converted_time}')
    else:
        print('Invalid state abbreviation.')


Original time (UTC): 2023-02-17 17:30
Converted time (CA): 2023-02-17 09:30:00-08:00


# Without PyTZ and US packages

In [3]:

from datetime import datetime
from dateutil import tz

def find_tz(location_state_abbr):
    state_timezones = {
        "AL": "America/Chicago",
        "AK": "America/Anchorage",
        "AZ": "America/Phoenix",
        "AR": "America/Chicago",
        "CA": "America/Los_Angeles",
        # Add the remaining states with their respective time zones
    }
    
    location_state_tz = state_timezones.get(location_state_abbr)
    
    if location_state_tz:
        return tz.gettz(location_state_tz)
    else:
        return None

def hopper_py(location_state_abbr, signing_time_str):
    signing_time = datetime.strptime(signing_time_str, "%Y-%m-%d %H:%M")
    state_timezone = find_tz(location_state_abbr)
    
    if state_timezone:
        signing_time_utc = signing_time.replace(tzinfo=tz.UTC)
        converted_time = signing_time_utc.astimezone(state_timezone)
        return converted_time.strftime("%Y-%m-%d %H:%M")
    else:
        return None

print(hopper_py('CA', '2023-02-17 17:30'))



2023-02-17 09:30


# Without US but with pytz (Current snowflake configuration)

In [19]:
from datetime import datetime
import pytz

from datetime import datetime
from dateutil import tz

def find_tz(location_state_abbr):
    state_timezones = {
        "AL": "America/Chicago",
        "AK": "America/Anchorage",
        "AZ": "America/Phoenix",
        "AR": "America/Chicago",
        "CA": "America/Los_Angeles",
        # Add the remaining states with their respective time zones
        "CO": "America/Denver",
        "CT": "America/New_York",
        "DE": "America/New_York",
        "FL": "America/New_York",
        "GA": "America/New_York",
        "HI": "Pacific/Honolulu",
        "ID": "America/Boise",
        "IL": "America/Chicago",
        "IN": "America/New_York",
        "IA": "America/Chicago",
        "KS": "America/Chicago",
        "KY": "America/New_York",
        "LA": "America/Chicago",
        "ME": "America/New_York",
        "MD": "America/New_York",
        "MA": "America/New_York",
        "MI": "America/New_York",
        "MN": "America/Chicago",
        "MS": "America/Chicago",
        "MO": "America/Chicago",
        "MT": "America/Denver",
        "NE": "America/Chicago",
        "NV": "America/Los_Angeles",
        "NH": "America/New_York",
        "NJ": "America/New_York",
        "NM": "America/Denver",
        "NY": "America/New_York",
        "NC": "America/New_York",
        "ND": "America/Chicago",
        "OH": "America/New_York",
        "OK": "America/Chicago",
        "OR": "America/Los_Angeles",
        "PA": "America/New_York",
        "RI": "America/New_York",
        "SC": "America/New_York",
        "SD": "America/Chicago",
        "TN": "America/Chicago",
        "TX": "America/Chicago",
        "UT": "America/Denver",
        "VT": "America/New_York",
        "VA": "America/New_York",
        "WA": "America/Los_Angeles",
        "WV": "America/New_York",
        "WI": "America/Chicago",
        "WY": "America/Denver"
    }
    
    location_state_tz = state_timezones.get(location_state_abbr)
    
    if location_state_tz:
        return tz.gettz(location_state_tz)
    else:
        return None
    
# Maybe add above function within this? Actually no. It can be modular this way
def time_hopper(location_state_abbr, signing_time_str):
    signing_time = datetime.strptime(signing_time_str, '%Y-%m-%d %H:%M')
    state_timezone = find_tz(location_state_abbr)
    if state_timezone:
        signing_time_utc = signing_time.replace(tzinfo=pytz.utc)
        converted_time = signing_time_utc.astimezone(state_timezone)
        return converted_time
    else:
        return None

if __name__ == '__main__':
    location_state_abbr = 'WA'
    signing_time_str = '2023-02-17 17:30'
    converted_time = time_hopper(location_state_abbr, signing_time_str)
    
    if converted_time:
        print(f'Original time (UTC): {signing_time_str}')
        print(f'Converted time ({location_state_abbr}): {converted_time}')
    else:
        print('Invalid state abbreviation.')

Original time (UTC): 2023-02-17 17:30
Converted time (WA): 2023-02-17 09:30:00-08:00


In [22]:
from datetime import datetime
import pytz
from dateutil import tz

def find_tz(location_state_abbr):
    state_timezones = {
        "AL": "America/Chicago",
        "AK": "America/Anchorage",
        "AZ": "America/Phoenix",
        "AR": "America/Chicago",
        "CA": "America/Los_Angeles",
        "CO": "America/Denver",
        "CT": "America/New_York",
        "DE": "America/New_York",
        "FL": "America/New_York",
        "GA": "America/New_York",
        "HI": "Pacific/Honolulu",
        "ID": "America/Boise",
        "IL": "America/Chicago",
        "IN": "America/New_York",
        "IA": "America/Chicago",
        "KS": "America/Chicago",
        "KY": "America/New_York",
        "LA": "America/Chicago",
        "ME": "America/New_York",
        "MD": "America/New_York",
        "MA": "America/New_York",
        "MI": "America/New_York",
        "MN": "America/Chicago",
        "MS": "America/Chicago",
        "MO": "America/Chicago",
        "MT": "America/Denver",
        "NE": "America/Chicago",
        "NV": "America/Los_Angeles",
        "NH": "America/New_York",
        "NJ": "America/New_York",
        "NM": "America/Denver",
        "NY": "America/New_York",
        "NC": "America/New_York",
        "ND": "America/Chicago",
        "OH": "America/New_York",
        "OK": "America/Chicago",
        "OR": "America/Los_Angeles",
        "PA": "America/New_York",
        "RI": "America/New_York",
        "SC": "America/New_York",
        "SD": "America/Chicago",
        "TN": "America/Chicago",
        "TX": "America/Chicago",
        "UT": "America/Denver",
        "VT": "America/New_York",
        "VA": "America/New_York",
        "WA": "America/Los_Angeles",
        "WV": "America/New_York",
        "WI": "America/Chicago",
        "WY": "America/Denver"
        }
    location_state_tz = state_timezones.get(location_state_abbr)
    if location_state_tz:
        return tz.gettz(location_state_tz)
    else:
        return None

def time_hopper(location_state_abbr, signing_time_str):
    signing_time = datetime.strptime(signing_time_str, "%Y-%m-%d %H:%M")
    state_timezone = find_tz(location_state_abbr)
    
    if state_timezone:
        signing_time_utc = signing_time.replace(tzinfo=tz.UTC)
        converted_time = signing_time_utc.astimezone(state_timezone)
        return converted_time.strftime("%Y-%m-%d %H:%M")
    else:
        return None


In [23]:
result = time_hopper(location_state_abbr, signing_time_str)
print(result)

2023-02-17 09:30
